In [ ]:
using CSV, DataFrames, Plots, StatsPlots, Dates, GLM;
using Pipe, StatsPlots, Underscores, Statistics, StatsBase, RollingFunctions;
# import data and clean
date_of_first_obs = Date("2020-07-01");
windowsize = 7; # set window size for aggregating data into windowsize-ly data
sub_sample_size = 90;  # 3-months
fixN_daily = 100000;  # to get adjusted case numbers
fixN_weekly = fixN_daily * 7;
fixN_7dd = fixN_daily;
Nstar = 5800000;
daily = CSV.File("pos.csv") |> DataFrame;
n = nrow(daily);
delete!(daily, n-1:n);  # last two rows contain some words instead of data, remomve them
daily = rename(daily, :Tested => :Nt, :NewPositive => :Ct) |> DataFrame; # rename Nt and Ct
daily.Nt .= replace.(daily.Nt, "." => "");  # There are dots in numbers, i.e. one thousand is written as 1.000, remove dots
daily.Nt .= replace.(daily.Nt, " " => "");
daily.Ct .= replace.(daily.Ct, "." => "");
daily.Ct .= replace.(daily.Ct, " " => "");
daily.Nt = parse.(Int, daily.Nt); # change data type from string to integers
daily.Ct = parse.(Int, daily.Ct);
daily.logNt = log.(daily.Nt); # create log(Nt)
daily.logCt = log.(daily.Ct); # create log(Ct)
daily.pihat = daily.Ct ./ daily.Nt; # calculate positive ratio
daily.pihat = replace!(daily.pihat, NaN => 0); # NaN are created bc some Nt=0, replace NaN with 0
daily.Date = Date.(daily.Date, "yyyy-mm-dd"); # change data type of Date from string to Date
n = nrow(daily);

# create daily data set 
national_daily = @pipe daily |> 
subset(_, :logCt => c -> .!isinf.(c)) |> 
select(_, :Date, :logCt, :logNt, :Ct, :Nt, :pihat) |>
transform(_, :logCt => (x -> x - lag(x)) => :diff_logCt, 
             :logNt => (x -> x - lag(x)) => :diff_logNt,
             :logNt => (x -> lag(x)) => :lag_logNt) |>
dropmissing(_) |>
subset(_, :Date => ByRow(D -> D >= date_of_first_obs)); # drop data before 2020-07-01 ;
N_daily = nrow(national_daily);

# create rolling weekly data set
national_weekly = select(daily,
    :Date,
    :pihat,
    :Nt => (v -> runmean(v, windowsize) * windowsize) => :Nt, 
    :Ct => (v -> runmean(v, windowsize) * windowsize) => :Ct)
delete!(national_weekly, 1: (windowsize - 1));
national_weekly = @pipe national_weekly |> 
transform(_, :Nt => (x -> log.(x)) => :logNt,
             :Ct => (x -> log.(x)) => :logCt,
             :pihat => (x -> log.(x)) => :logPi) |> 
subset(_, :logCt => c -> .!isinf.(c)) |> 
transform(_, :logCt => (x -> x - lag(x)) => :diff_logCt, 
             :logNt => (x -> x - lag(x)) => :diff_logNt,
             :logPi => (x -> x - lag(x)) => :diff_logPi,
             :logNt => (x -> lag(x)) => :lag_logNt) |> 
dropmissing(_) |>
subset(_, :Date => ByRow(D -> D >= date_of_first_obs)); # drop data before 2020-07-01 ;
N_weekly = nrow(national_weekly);

# create weekday data set, 7-day difference
national_7dd = @pipe daily |> 
subset(_, :logCt => c -> .!isinf.(c)) |> 
select(_, :Date, :logCt, :logNt, :Ct, :Nt, :pihat) |>
transform(_, :logCt => (x -> x - lag(x, 7)) => :diff_logCt, 
             :logNt => (x -> x - lag(x, 7)) => :diff_logNt,
             :logNt => (x -> lag(x)) => :lag_logNt) |>
dropmissing(_) |>
subset(_, :Date => ByRow(D -> D >= date_of_first_obs)); # drop data before 2020-07-01 ;
N_7dd = nrow(national_7dd);

# add sub-sample index to data sets
national_daily.subsample = string.(Int.(repeat(1:ceil(N_daily/sub_sample_size), 
            inner = sub_sample_size)[1:N_daily]));
national_weekly.subsample = string.(Int.(repeat(1:ceil(N_weekly/sub_sample_size), 
            inner = sub_sample_size)[1:N_weekly]));
national_7dd.subsample = string.(Int.(repeat(1:ceil(N_7dd/sub_sample_size), 
            inner = sub_sample_size)[1:N_7dd]));
national_weekly.C_tilde = national_weekly.Ct ./ national_weekly.Nt .* Nstar;


# Old text

Define $N_t$ as number of test conducted at day $t$;
       $C_t$ as number of observed positive cases at day $t$;
       $H_t$ as newly admitted to hospitals at day $t$;
       $D_t$ as new death at day $t$.
       
Also define two unobserved variables, $C_t^*$ as the true number of cases at day $t$ (propotional to the whole population); $S_t^*$ as the severity which is related to the case fatality ratio or the hospitality ratio. 

Since $C_t^*$ is not affected by $N_t$, but $C_t$ is. To be more specific, when we double number of testing at day $t$, i.e. increase $N_t$ to $2 N_t$, the observed number of postive cases will also increase, although the increment is likely to be less than one $C_t$. So, $C_t$ by itself is not a good estimator for $C_t^*$. Therefore, we want to take into account the variable $N_t$ and construct a more reliable estimate for $C_t^*$.

A model proposed,

$\hat{C_t^*} = C_t \cdot \left( \frac{N}{N_t} \right)^{\beta}$

where $\beta > 0$. 

Take log transformation, we have

$\log \hat{C_t^*}= \log C_t + \beta \cdot \log \frac{N}{N_t}$

After taking take first difference, we get

$\Delta \log \hat{C_t^*} = \Delta \log C_t - \beta \cdot \Delta \log N_t$.

Rearrange the model, we can get the OLS estimate of $\beta$:

$\Delta \log C_t = \beta \cdot \Delta \log N_t + \Delta \log \hat{C_t^*}$,

$\hat{\beta} = \frac{\sum \Delta \log N_t  \Delta \log C_t}{\sum \left( \Delta \log N_t\right)^2}$.

In [ ]:
# different initial values result in different results 
 θ_national_weekly = [0, 1, 0.1, 2, 0];
# θ_national_weekly = [0, 0.1, 0.1, 0.9, 0];
obj_national_weekly = θ_national_weekly -> logL(θ_national_weekly, national_weekly) 
res_national_weekly = optimize(obj_national_weekly, θ_national_weekly)
Optim.minimizer(res_national_weekly)  # Estimated parameters

In [ ]:
ScoreBetaHat_national_weekly = ScoreBeta(Optim.minimizer(res_national_weekly), national_weekly);
plot(national_weekly.Date, ScoreBetaHat_national_weekly, 
label = :none,
xlabel = "First day of each rolling week",
ylabel = "Score \\beta_t")
hline!([0], color = :red, label = :none)
hline!([1], color = :red, label = :none)

In [ ]:
national_weekly.ScoreBetaHat = ScoreBetaHat_national_weekly;
national_weekly.C_tilde_score = exp.(national_weekly.logCt .- 
national_weekly.ScoreBetaHat .* national_weekly.logNt .+
national_weekly.ScoreBetaHat .* nstar)
@df national_weekly plot(:Date, :C_tilde_score, yaxis = :log,
    label = "Score Adjusted log(Ct)", xlabel = "First day of each rolling week", ylabel = "Ct values", color = :red)
@df national_weekly plot!(:Date, :Ct, yaxis = :log,
    label = "Observed log(Ct)", leg = :topleft, color = :blue)
#@df national_weekly plot!(twinx(),:Date, :Nt, yaxis = :log,
#    label = "Observed log(Nt)", leg = :bottomright, color = :black)

# Old score model
Assume normal errors, $\Delta \log \hat{C_t^*} \sim N(0, \sigma^2)$. 

Simplify notation, $y_t = \Delta \log C_t$ and $x_t = \Delta \log N_t$.

The log-likelihood is:

$$\ell(\beta_t) \propto - T \log(\sigma^2) - \sum_{t=1}^T \frac{(y_t - \beta_t x_t)^2}{\sigma^2};$$

The score is:

$$\frac{\partial \ell}{\partial \beta_t} = \frac{1}{\sigma^2} (y_t - \beta_t x_t) x_t;$$

The hessian is:

$$\frac{\partial^2 \ell}{\partial \beta_t^2} = - \frac{1}{\sigma^2} x_t^2;$$

Define
$$\psi(\beta_t) =
\frac{\frac{\partial \ell}{\partial \beta_t}}{\sqrt{- \frac{\partial^2 \ell}{\partial \beta_t^2}}}
=
\frac{\frac{1}{\sigma^2} (y_t - \beta_t x_t) x_t}{\sqrt{\frac{1}{\sigma^2} x_t^2}}
= 
\frac{1}{\sigma} \text{sign}(x_t) (y_t - \beta_t x_t)
$$

The model is (new version):

$$\beta_{t+1} = \omega + \phi \beta_t + \alpha \psi(\beta_t)
=
\omega + \phi \beta_t + \alpha \frac{\text{sign}(x_t) (y_t - \beta_t x_t)}{\sigma} 
$$

In [ ]:
step = 100;  # choose sample size to be forecasted
function ScoreOutFcast(step, data)
    n = nrow(data);
    yhat = [];
    θ = [0, 1, 1, 0.2, 0.6];
    for i in 1:step
        ss = n - step + i - 1;
        train = data[1:ss, :];
        x = train.diff_logNt;
        y = train.diff_logCt;
        obj = θ -> logL(θ, train);
        res = optimize(obj, θ);
        θ = Optim.minimizer(res);
        β =  recoverBeta(θ, train);
        ω = θ[1]; 
        ϕ = θ[2];
        α = θ[3];
        σ = θ[4];
        βt1 = ω + ϕ * β[ss] + α * sign(x[ss]) * (y[ss] - β[ss] * x[ss]) / σ;
        xt1 = data.diff_logNt[ss + 1];
        yt1hat = βt1 * xt1;
        push!(yhat, yt1hat)
    end
    return yhat
end    
ScoreYHat_national_daily = ScoreOutFcast(step, national_daily);
ScoreYHat_national_weekly = ScoreOutFcast(step, national_weekly);
ScoreYHat_national_7dd = ScoreOutFcast(step, national_7dd);

## Section 2.3: Score Out-of-sample Forecast<a class="anchor" id="dynamic_out_sample"></a>

The name "step" in my code means the sample size of data kept for out-of-sample forecast.

If step = 10, I estimate $\beta$ for 10 times. The first time, I use all data but the last 10 for estimation, and then make one out-of-sample forecast for the last 10th observation; The second time, I use all data but the last 9 for estimation, and only make one step ahead forecast for the last 9th observation; ... Repeat until I have all out-of-sample forecast for the 10 observations.

The graph shows observed $\Delta log C_t$ in blue and predicted in red. 

In [ ]:
step = 10;  # choose sample size to be forecasted
function ScoreOutFcast(step, data)
    n = nrow(data);
    yhat = [];
    θstatic0 = [0, 0.1, 1, 0.2, 0.6];
    for i in 1:step
        ss = n - step + i - 1;
        train = data[1:ss, :];
        x = train.diff_logNt;
        y = train.diff_logCt;
        obj = θ -> logL(θ, train);
        res = optimize(obj, θstatic0);
        θstatic = Optim.minimizer(res);
        β =  recoverBeta(θstatic, train);
        θ = log.(β ./ (1 .- β));
        ω = θstatic[1]; 
        ϕ = θstatic[2];
        α = θstatic[3];
        σ = θstatic[4];
        score = x[ss] / (σ^2 * exp(θ[ss]) * (1 + exp(-θ[ss]))^2) * (y[ss] - x[ss] / (1 + exp(-θ[ss])));
        hessian = x[ss] / (σ^2 * exp(θ[ss]) * (1 + exp(-θ[ss]))^2) *
        (2 * y[ss] / (exp(θ[ss]) * (1 + exp(-θ[ss]))) - 
            y[ss] - 
            3 * x[ss] / (exp(θ[ss]) * (1 + exp(-θ[ss]))^2) + 
            x[ss] /  (1 + exp(-θ[ss])));
        psi = score / sqrt(abs(hessian));
        newθ = ω + ϕ * θ[ss] + α * psi;
        βt1 = 1/( 1 + exp(-newθ) );
        xt1 = data.diff_logNt[ss + 1];
        yt1hat = βt1 * xt1;
        push!(yhat, yt1hat)
    end
    return yhat
end    
ScoreYHat_national_daily = ScoreOutFcast(step, national_daily);
ScoreYHat_national_weekly = ScoreOutFcast(step, national_weekly);
ScoreYHat_national_7dd = ScoreOutFcast(step, national_7dd);

In [ ]:
# daily
plot(national_daily.Date[N_daily - step + 1: N_daily], 
    national_daily.diff_logCt[N_daily - step + 1: N_daily],
    color = :blue,
    xlabel = "Date",
    ylabel = "\\Delta log Ct",
    label = "real")
plot!(national_daily.Date[N_daily - step + 1: N_daily], 
    ScoreYHat_national_daily,
    color = :red,
    label = "fitted", leg = :topleft)

In [ ]:
# weekly
plot(national_weekly.Date[N_weekly - step + 1: N_weekly], 
    national_weekly.diff_logCt[N_weekly - step + 1: N_weekly],
    color = :blue,
    xlabel = "Date",
    ylabel = "\\Delta log Ct",
    label = "real")
plot!(national_weekly.Date[N_weekly - step + 1: N_weekly], 
    ScoreYHat_national_weekly,
    color = :red,
    label = "fitted", leg = :bottomleft)

In [ ]:
# weekday
plot(national_7dd.Date[N_7dd - step + 1: N_7dd], 
    national_7dd.diff_logCt[N_7dd - step + 1: N_7dd],
    color = :blue,
    xlabel = "Date",
    ylabel = "\\Delta log Ct",
    label = "real")
plot!(national_7dd.Date[N_7dd - step + 1: N_7dd], 
    ScoreYHat_national_7dd,
    color = :red,
    label = "fitted", leg = :bottomleft)

## Section 3.2: Dynamic Analysis
First restrict the intercept and slope in the socre equation to be 0 and 1 respectively.

The $\beta_0$ is also part of the parameters now.

In [ ]:
# Restrict ω = 0, and ϕ = 1.
function Restricted_ScoreBeta(θ, data)
    x = data.diff_logNt;
    y = data.diff_logCt;
    T = nrow(data);
    ω = 0;
    ϕ = 1;   
    α = θ[1]; 
    σ = θ[2];
    β0 = θ[3];
    β = [];
    push!(β, β0)  
    
    for i in 1:(T-1)
    v = ω + ϕ * β[i] + α * sign(x[i]) * (y[i] - β[i] * x[i]) / σ
    push!(β, v)
    end
    
    return β
end

function Restricted_logL(θ, data)
    x = data.diff_logNt;
    y = data.diff_logCt;
    T = nrow(data);
    β = Restricted_ScoreBeta(θ, data);
    σ = θ[2];
    sum((y .- β .* x).^2)/(σ^2) + T*log(σ^2)   # Note: This is negative logL, not logL.
end

In [ ]:
muni_daily_score_coef_restricted = DataFrame(City = String[], α = String[], σ = String[], β0 = String[]);
muni_daily_score_beta_restricted = DataFrame();
θ = [1, 0.2, 0.6];
for i in 1 : Int((ncol(muni_daily)-1)/2)
    diff_logCt = muni_daily[:, i + 1]
    # to match Ct and Nt for the same city ---->
    city_name = names(muni_daily)[i + 1]  
    diff_logNt = muni_daily[:, string(city_name, "_1")]
    data = DataFrame(diff_logCt = diff_logCt, diff_logNt = diff_logNt)
    obj = θ -> Restricted_logL(θ, data);
    res = optimize(obj, θ);
    coef_w_name = @pipe Optim.minimizer(res) |> round.(_, digits = 4) |> string.(_) |> append!([city_name],  _);
    push!(muni_daily_score_coef_restricted, coef_w_name);
    muni_daily_score_beta_restricted[!, city_name] = Restricted_ScoreBeta(Optim.minimizer(res), data);
end
insertcols!(muni_daily_score_beta_restricted, 1, :Date => muni_daily.Date);

In [ ]:
muni_weekly_score_coef_restricted = DataFrame(City = String[], α = String[], σ = String[], β0 = String[]);
muni_weekly_score_beta_restricted = DataFrame();
θ = [1, 0.2, 0.6];
for i in 1 : Int((ncol(muni_weekly)-1)/2)
    diff_logCt = muni_weekly[:, i + 1]
    # to match Ct and Nt for the same city ---->
    city_name = names(muni_weekly)[i + 1]  
    diff_logNt = muni_weekly[:, string(city_name, "_1")]
    data = DataFrame(diff_logCt = diff_logCt, diff_logNt = diff_logNt)
    obj = θ -> Restricted_logL(θ, data);
    res = optimize(obj, θ);
    coef_w_name = @pipe Optim.minimizer(res) |> round.(_, digits = 4) |> string.(_) |> append!([city_name],  _);
    push!(muni_weekly_score_coef_restricted, coef_w_name);
    muni_weekly_score_beta_restricted[!, city_name] = Restricted_ScoreBeta(Optim.minimizer(res), data);
end
insertcols!(muni_weekly_score_beta_restricted, 1, :Date => muni_weekly.Date);

In [ ]:
muni_7dd_score_coef_restricted = DataFrame(City = String[], α = String[], σ = String[], β0 = String[]);
muni_7dd_score_beta_restricted = DataFrame();
θ = [1, 0.2, 0.6];
for i in 1 : Int((ncol(muni_7dd)-1)/2)
    diff_logCt = muni_7dd[:, i + 1]
    # to match Ct and Nt for the same city ---->
    city_name = names(muni_7dd)[i + 1]  
    diff_logNt = muni_7dd[:, string(city_name, "_1")]
    data = DataFrame(diff_logCt = diff_logCt, diff_logNt = diff_logNt)
    obj = θ -> Restricted_logL(θ, data);
    res = optimize(obj, θ);
    coef_w_name = @pipe Optim.minimizer(res) |> round.(_, digits = 4) |> string.(_) |> append!([city_name],  _);
    push!(muni_7dd_score_coef_restricted, coef_w_name);
    muni_7dd_score_beta_restricted[!, city_name] = Restricted_ScoreBeta(Optim.minimizer(res), data);
end
insertcols!(muni_7dd_score_beta_restricted, 1, :Date => muni_7dd.Date);

In [ ]:
# Abandon code
# Aggregate daily data to weekly data (non-overlap, non-rolling window)
# df = @pipe daily |> select(_, :Ct, :Nt);
# df.week = repeat((1:ceil(Int, n/7)), inner = 7)[1:n];
# weekly =  @_ groupby(df, [:week]) |> combine(__, :Ct => sum => :WeekCt, :Nt => sum => :WeekNt);
# weekly.logCt = log.(weekly.WeekCt);
# weekly.logNt = log.(weekly.WeekNt);
# weekly = @pipe transform(weekly, :logCt => (x -> x - lag(x)) => :diff_logCt, 
#             :logNt => (x -> x - lag(x)) => :diff_logNt) |> 
# dropmissing(_) |> 
# subset(_, :diff_logCt => c -> .!isinf.(c), :diff_logCt => c -> .!isnan.(c));